# DRUG CONSUMPTION - GRUPO 1

Realizado por:
* **Alejandro López-Fando Fernández**  
* **Jerson Peña Martínez**  
* **Manuela Larrea**

Versiones:
* Versión Python: **3.11.5**  
* Versión Pandas: **2.0.3**  
* Versión PlotNine: **0.12.4**

In [1]:
import sys
import pandas as pd
import utilities.constants as cte
import plotnine as p9

En la carpeta utilities se encuentra el script constants.py con constantes, esta manera de organizar las cosas hará que el código quede más limpio.  
Leamos el set de datos descargado de la siguiente url:  
https://archive.ics.uci.edu/dataset/373/drug+consumption+quantified

In [2]:
df = pd.read_csv('../data/drug_consumption.data', sep=',', header=None, names=cte.names)

Transformamos las columnas para ser capaces de entender el set de datos. Para la transformación seguimos el guión que nos proponen en la url.

In [3]:
df.age = df.age.apply(lambda row: cte.age_values[row])
df.gender = df.gender.apply(lambda row: cte.gender_values[row])
df.education = df.education.apply(lambda row: cte.education_values[row])
df.country = df.country.apply(lambda row: cte.country_values[row])
df.ethnicity = df.ethnicity.apply(lambda row: cte.ethnicity_values[row])
df.n_score = df.n_score.apply(lambda row: cte.nscore_values[row])
df.e_score = df.e_score.apply(lambda row: cte.escore_values[row])
df.o_score = df.o_score.apply(lambda row: cte.oscore_values[row])
df.a_score = df.a_score.apply(lambda row: cte.ascore_values[row])
df.c_score = df.c_score.apply(lambda row: cte.cscore_values[row])
for droga in cte.all_drugs_columns:
    df[droga] = df[droga].apply(lambda row: cte.drugs_values[row])

Hagamos un primer exploratorio de nuestro DataFrame.

¿Cuántas filas y columnas tenemos?

In [4]:
df.shape

(1885, 32)

¿Tenemos valores nulos?

In [5]:
df.isna().sum()

id                  0
age                 0
gender              0
education           0
country             0
ethnicity           0
n_score             0
e_score             0
o_score             0
a_score             0
c_score             0
impulsive           0
sensation_seeing    0
alcohol             0
amphet              0
amyl                0
benzos              0
caffeine            0
cannabis            0
chocolate           0
cocaine             0
crack               0
ecstasy             0
heroin              0
ketamine            0
legal_highs         0
lsd                 0
methadone           0
mushrooms           0
nicotine            0
semer               0
vsa                 0
dtype: int64

Observamos que en ninguna columna tenemos valores nulos.  
¿El tipop de nuestras columnas es el adecuado?

In [6]:
df.dtypes

id                    int64
age                  object
gender               object
education            object
country              object
ethnicity            object
n_score               int64
e_score               int64
o_score               int64
a_score               int64
c_score               int64
impulsive           float64
sensation_seeing    float64
alcohol              object
amphet               object
amyl                 object
benzos               object
caffeine             object
cannabis             object
chocolate            object
cocaine              object
crack                object
ecstasy              object
heroin               object
ketamine             object
legal_highs          object
lsd                  object
methadone            object
mushrooms            object
nicotine             object
semer                object
vsa                  object
dtype: object

En principio, salvo el Id que estaría mejor siendo de tipo object, el resto de columnas tienen buena pinta. Veamos el Id.

In [7]:
df.id.head()

0    1
1    2
2    3
3    4
4    5
Name: id, dtype: int64

La columna Id no es más que un índice y, para lo que vamos a hacer en este trabajo, no nos aporta nada luego decidimos borrar esta columna.

In [8]:
del df['id']

La 'droga' Semeron es una droga ficticia que se inventaron los que hicieron el set de datos para identificar mentirosos, veamos cuanta gente ha dicho que alguna vez consumió esta droga inventada.

In [9]:
df.loc[df.semer != 'Never Used', 'semer'].count()

8

Vemos que 8 personas han mentido como mínimo diciendo que han consumido Semeron, al ser solo 8 registros y por si acaso han mentido en algún otro campo decidimos borrar estos registros de nuestro DataFrame. Por tanto también eliminamos la columna Semeron de nuestro DataFrame ya que ya no proporciona información.

In [10]:
df = df.loc[df.semer == 'Never Used'].reset_index(drop=True)
del df['semer']

Busquemos ahora outliers o datos que sean un poco extraños, por ejemplo gente que esté entre 18-24 años y hayan consumido alguna droga hace más de 10 años.

In [11]:
for droga in cte.drugs_columns:
    n = df.loc[(df.age == '18-24') & (df[droga] == 'Used over a Decade Ago'), droga].count()
    print(f'Hay {n} personas menores de 24 años que han consumido {droga} hace más de 10 años.')

Hay 0 personas menores de 24 años que han consumido alcohol hace más de 10 años.
Hay 1 personas menores de 24 años que han consumido amphet hace más de 10 años.
Hay 2 personas menores de 24 años que han consumido amyl hace más de 10 años.
Hay 3 personas menores de 24 años que han consumido benzos hace más de 10 años.
Hay 1 personas menores de 24 años que han consumido caffeine hace más de 10 años.
Hay 3 personas menores de 24 años que han consumido cannabis hace más de 10 años.
Hay 0 personas menores de 24 años que han consumido chocolate hace más de 10 años.
Hay 3 personas menores de 24 años que han consumido cocaine hace más de 10 años.
Hay 1 personas menores de 24 años que han consumido crack hace más de 10 años.
Hay 0 personas menores de 24 años que han consumido ecstasy hace más de 10 años.
Hay 0 personas menores de 24 años que han consumido heroin hace más de 10 años.
Hay 0 personas menores de 24 años que han consumido ketamine hace más de 10 años.
Hay 1 personas menores de 24 añ

In [12]:
df.loc[(df.age == '18-24') & (df['vsa'] == 'Used over a Decade Ago')]

,age,gender,education,country,ethnicity,n_score,e_score,o_score,a_score,c_score,...,crack,ecstasy,heroin,ketamine,legal_highs,lsd,methadone,mushrooms,nicotine,vsa
543,18-24,Female,Left school at 16 years,UK,White,48,36,41,38,34,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used in Last Day,Used over a Decade Ago
565,18-24,Female,University degree,UK,White,45,47,39,49,46,...,Never Used,Used in Last Decade,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used in Last Day,Used over a Decade Ago
846,18-24,Male,Masters degree,Other,White,40,33,49,41,45,...,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Never Used,Used in Last Month,Used in Last Day,Used over a Decade Ago
851,18-24,Male,Left school at 16 years,Canada,Mixed-White/Black,41,45,50,36,33,...,Used in Last Month,Used in Last Year,Used in Last Week,Used in Last Decade,Never Used,Used in Last Decade,Used in Last Month,Used in Last Decade,Used in Last Day,Used over a Decade Ago
860,18-24,Male,"Some college or university, no certificate or ...",Republic of Ireland,White,51,36,45,32,37,...,Never Used,Used in Last Year,Never Used,Never Used,Used in Last Day,Used over a Decade Ago,Used in Last Year,Used in Last Decade,Used in Last Day,Used over a Decade Ago
962,18-24,Female,"Some college or university, no certificate or ...",USA,White,30,42,50,50,44,...,Never Used,Never Used,Never Used,Never Used,Used in Last Year,Never Used,Used over a Decade Ago,Used in Last Year,Used in Last Month,Used over a Decade Ago
1029,18-24,Female,"Some college or university, no certificate or ...",UK,White,48,36,49,39,34,...,Never Used,Used in Last Month,Never Used,Never Used,Used in Last Month,Never Used,Never Used,Never Used,Used in Last Day,Used over a Decade Ago
1042,18-24,Male,"Some college or university, no certificate or ...",USA,White,48,31,49,43,44,...,Never Used,Used in Last Decade,Never Used,Used in Last Decade,Used in Last Decade,Used in Last Decade,Used in Last Year,Never Used,Used in Last Year,Used over a Decade Ago
1080,18-24,Female,University degree,UK,White,35,39,49,45,45,...,Never Used,Used in Last Year,Never Used,Used in Last Decade,Used in Last Year,Never Used,Never Used,Used in Last Decade,Used in Last Decade,Used over a Decade Ago
1359,18-24,Male,"Some college or university, no certificate or ...",UK,White,44,44,40,36,31,...,Never Used,Used in Last Week,Never Used,Used in Last Year,Used in Last Year,Never Used,Used in Last Week,Never Used,Used in Last Day,Used over a Decade Ago


Al haber pocos registros de gente que haya consumido droga siendo muy jóvenes podemos creernos que son excepciones y son reales, por tanto decidimos mantener estos registros. Observamos que 16 personas consumieron vsa siendo muy jóvenes, pero, tras informarnos, vimos que oler pegamento se puede considerar en esta categoría y, por tanto, mantenemos también estos 16 registros.

Hagamos un exploratorio un poco más profundo de nuestras columnas, el objetivo es agrupar los valores que puede tomar cada columna para tener mayor representatividad de cada categoría. Además conseguiremos tener menos categorías en cada columna y así poder sacar insights y gráficos más claros.

Empecemos por la columna de la edad.

In [13]:
cont_reg_valor = df['age'].value_counts()
porcen_reg_valor = (cont_reg_valor / df.shape[0]) * 100
pd.DataFrame({'n_registros' : cont_reg_valor, 'porcentaje' : porcen_reg_valor})

,n_registros,porcentaje
age,,
18-24,637,33.937134
25-34,480,25.572722
35-44,355,18.913159
45-54,294,15.663292
55-64,93,4.954715
65+,18,0.958977


Para la columna age decidimos agrupar el grupo de personas con 65+ años con el grupo de personas de 55-64 años ya que tenemos menos de un 1% de personas con 65+ años y son dos grupos de población parecidos. El nuevo grupo sera 55+.

In [14]:
df.age = df.age.apply(lambda row: '55+' if row in ['55-64', '65+'] else row)

Veamos ahora la columna del género.

In [15]:
cont_reg_valor = df['gender'].value_counts()
porcen_reg_valor = (cont_reg_valor / df.shape[0]) * 100
pd.DataFrame({'n_registros' : cont_reg_valor, 'porcentaje' : porcen_reg_valor})

,n_registros,porcentaje
gender,,
Male,940,50.079915
Female,937,49.920085


La columna género está muy bien ya que tenemos 50% de hombres y 50% de mujeres, por lo que esta columna no la tocamos.

Sigamos con la columna de educación.

In [16]:
cont_reg_valor = df['education'].value_counts()
porcen_reg_valor = (cont_reg_valor / df.shape[0]) * 100
pd.DataFrame({'n_registros' : cont_reg_valor, 'porcentaje' : porcen_reg_valor})

,n_registros,porcentaje
education,,
"Some college or university, no certificate or degree",503,26.798082
University degree,478,25.466169
Masters degree,283,15.077251
Professional certificate/diploma,270,14.384656
Left school at 18 years,99,5.274374
Left school at 16 years,98,5.221097
Doctorate degree,89,4.741609
Left school at 17 years,29,1.545019
Left school before 16 years,28,1.491742


Para la columna education agrupamos las personas que abandonaron la escuela antes del los 16, las que la abandonaron a los 16, a los 17 y a los 18 como personas sin estudios, y agrupamos las personas con un grado universitario/master/doctorado como personas con estudios superiores. Los otros dos grupos lo forman estudiantes y personas con un título de formación profesional respectivamente. Así conseguimos tener 4 grupos de personas que más o menos tienen una educación similar, al pasar de 9 grupos a 4 conseguimos tener mayor representatividad en los grupos y que los gráficos sean más claros.

In [17]:
df.education = df.education.apply(lambda row: cte.education_values_grouped[row])

Veamos ahora la columna de país.

In [18]:
cont_reg_valor = df['country'].value_counts()
porcen_reg_valor = (cont_reg_valor / df.shape[0]) * 100
pd.DataFrame({'n_registros' : cont_reg_valor, 'porcentaje' : porcen_reg_valor})

,n_registros,porcentaje
country,,
UK,1044,55.620671
USA,551,29.355354
Other,118,6.286628
Canada,87,4.635056
Australia,52,2.770378
Republic of Ireland,20,1.065530
New Zealand,5,0.266383


En la columna country solo tenemos buena representación para UK y USA, sin embargo no vamos a borrar registros ya que aunque los registros de New Zealand no nos sirvan para hacer gráficos por país, sí son útiles para hacer gráficos por género/edad/educación...  
Lo que haremos para hacer gráficos por país será comparar UK y USA, por ello creamos un DataFrame con las personas que son de Uk o USA.

In [19]:
df_uk_usa = df.loc[(df.country == 'UK') | (df.country == 'USA')]

Sigamos con la columna de etnia.

In [20]:
cont_reg_valor = df['ethnicity'].value_counts()
porcen_reg_valor = (cont_reg_valor / df.shape[0]) * 100
pd.DataFrame({'n_registros' : cont_reg_valor, 'porcentaje' : porcen_reg_valor})

,n_registros,porcentaje
ethnicity,,
White,1715,91.369206
Other,62,3.303143
Black,33,1.758125
Asian,25,1.331913
Mixed-White/Asian,20,1.065530
Mixed-White/Black,19,1.012254
Mixed-Black/Asian,3,0.159830


Como vemos más del 90% de nuestros registros son gente blanca y del resto de etnias no tenemos apenas representatividad, por tanto decidimos que no sacaremos ningún insight por etnia, luego borramos la columna.

In [21]:
del df['ethnicity']

Respecto a las drogas que tenemos en nuestro DataFrame haremos lo siguiente:

1- Para cada droga vamos a transformar las 7 categorías (Never Used, Used over a Decade Ago, Used in Last Decade, Used in Last Year, Used in Last Month, Used in Last Week, Used in Last Day) en 2, lo haremos de la siguiente forma:  
* Consume (1) -> Formado por Used in Last Year, Used in Last Month, Used in Last Week y Used in Last Day  
* No consume (0) -> Formado por Never Used, Used over a Decade Ago y Used in Last Decade

In [22]:
for droga in cte.drugs_columns:
    df[droga] = df[droga].apply(lambda row: cte.drugs_values_grouped[row])

2- Eliminamos el chocolate ya que no es una droga como tal y no nos aporta nada para extraer algún insight.

In [23]:
del df['chocolate']

3- Como tenemos demasiadas drogas y muchas tienen efectos/usos similares vamos a agrupar en las siguientes categorías:  
* Alcohol.  
* Nicotina.  
* Cafeína.  
* Drogas recreacionales, formadas por cannabis, éxtasis, LSD y hongos.  
* Drogas estimulantes, formadas por anfetaminas, cocaína, metanfetamina.  
* Drogas sedantes, formadas por benzodiazepina, heroína, ketamina y legal highs.  
* Drogas de alto riesgo, formadas aminoácidos, crack y VSA.

In [24]:
df['recreational_drugs'] = df[['cannabis', 'ecstasy', 'lsd', 'mushrooms']].max(axis=1)
df['stimulant_drugs'] = df[['amphet', 'cocaine', 'methadone']].max(axis=1)
df['sedative_drugs'] = df[['benzos', 'heroin', 'ketamine', 'legal_highs']].max(axis=1)
df['high_risk_drugs'] = df[['amyl', 'crack', 'vsa']].max(axis=1)

Borramos los datos de las drogas por separado.

In [27]:
for droga in cte.drugs_to_delete:
    del df[droga]

In [28]:
df

,age,gender,education,country,n_score,e_score,o_score,a_score,c_score,impulsive,sensation_seeing,alcohol,caffeine,nicotine,recreational_drugs,stimulant_drugs,sedative_drugs,high_risk_drugs
0,35-44,Female,Estudios de FP,UK,39,36,42,37,42,-0.21712,-1.18084,1,1,0,0,0,0,0
1,25-34,Male,Estudios superiores,UK,29,52,55,48,41,-0.71126,-0.21575,1,1,1,1,1,0,0
2,35-44,Male,Estudios de FP,UK,31,45,40,32,34,-1.37983,0.40148,1,1,0,1,0,0,0
3,18-24,Female,Estudios superiores,UK,34,34,46,47,46,-1.37983,-1.18084,1,1,0,0,0,1,0
4,35-44,Female,Estudios superiores,UK,43,28,43,41,50,-0.21712,-0.21575,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872,18-24,Female,Estudiando,USA,25,51,57,48,33,0.88113,1.92173,1,1,0,1,0,1,1
1873,18-24,Male,Estudiando,USA,33,51,50,48,30,0.88113,0.76540,1,1,1,1,1,1,0
1874,25-34,Female,Estudios superiores,USA,47,30,37,31,31,0.52975,-0.52593,1,1,1,1,1,1,1
1875,18-24,Female,Estudiando,USA,45,26,48,32,22,1.29221,1.22470,1,1,1,1,0,1,0
